In [ ]:
import os
import zipfile
import glob
import pandas as pd
import numpy as np

# Display full DataFrame content (optional)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Paths
zip_file_path = '/drive/tennis_data.zip'
extract_path = 'extracted_data'
parquet_dir = 'parquet_data'

# Extract zip 
if not os.path.exists(zip_file_path):
    print(f"The file {zip_file_path} does not exist.")
else:
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
        print(f"Files extracted to {extract_path}")

# Convert CSV to Parquet
if not os.path.exists(parquet_dir):
    os.makedirs(parquet_dir)

for filename in os.listdir(extract_path):
    if filename.endswith('.csv'):
        csv_path = os.path.join(extract_path, filename)
        df = pd.read_csv(csv_path)
        parquet_path = os.path.join(parquet_dir, filename.replace('.csv', '.parquet'))
        df.to_parquet(parquet_path)


In [ ]:
def load_parquet_files(base_path, subfolder):
    files = []
    for folder in ['20240201', '20240202', '20240203']:
        path_pattern = os.path.join(base_path, 'data', folder, 'data', 'raw', subfolder, '*.parquet')
        files.extend(glob.glob(path_pattern))

    if not files:
        print(f"No files found for subfolder: {subfolder}")
        return None

    dfs = []
    for file in files:
        try:
            df = pd.read_parquet(file)
            dfs.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    return pd.concat(dfs, ignore_index=True) if dfs else None


In [ ]:
def load_all_datasets(base_path):
    print("Loading tennis datasets...")

    matches_df = load_parquet_files(base_path, 'raw_match_parquet')
    stats_df = load_parquet_files(base_path, 'raw_statistics_parquet')
    power_df = load_parquet_files(base_path, 'raw_tennis_power_parquet')
    votes_df = load_parquet_files(base_path, 'raw_votes_parquet')
    odds_df = load_parquet_files(base_path, 'raw_odds_parquet')
    pbp_df = load_parquet_files(base_path, 'raw_point_by_point_parquet')

    required_dfs = {
        "matches_df": matches_df,
        "stats_df": stats_df,
        "power_df": power_df
    }
    for name, df in required_dfs.items():
        if df is None or df.empty:
            raise ValueError(f"Dataset '{name}' failed to load or is empty.")

    print("All datasets loaded successfully.")
    return matches_df, stats_df, power_df, votes_df, odds_df, pbp_df

# Adjust base path if needed
base_path = os.path.join(os.path.expanduser('~'), 'Downloads', 'tennis_data')

matches_df, stats_df, power_df, votes_df, odds_df, pbp_df = load_all_datasets(base_path)


## Tennis Data Analysis: Questions 1–17

### 1. How many tennis players are included in the dataset?

In [ ]:

# Combine all unique player names from match dataset
players = pd.concat([matches_df['player1'], matches_df['player2']]).unique()
print(f"Total unique players: {len(players)}")


### 2. What is the average height of the players?

In [ ]:

# Check height in power_df
average_height = power_df['player_height'].dropna().mean()
print(f"Average player height: {average_height:.2f} cm")


In [ ]:
# Q1: Total number of matches
def q1(matches_df):
    return len(matches_df)

# Q2: Number of unique players
def q2(matches_df):
    return pd.unique(matches_df[['winner_id', 'loser_id']].values.ravel()).size

# Q3: Top 5 players with most wins
def q3(matches_df):
    return matches_df['winner_name'].value_counts().head(5)

# Q4: Top 5 players with most losses
def q4(matches_df):
    return matches_df['loser_name'].value_counts().head(5)

# Q5: Average match duration
def q5(matches_df):
    if 'match_duration' in matches_df.columns:
        return matches_df['match_duration'].mean()
    else:
        return "Column 'match_duration' not found in matches_df."

# Q6: Surface distribution of matches
def q6(matches_df):
    return matches_df['surface'].value_counts()

# Q7: Distribution of match outcomes by tournament level
def q7(matches_df):
    return matches_df['tourney_level'].value_counts()

# Q8: Player with highest average aces per match
def q8(stats_df):
    if 'player_id' in stats_df.columns and 'aces' in stats_df.columns:
        aces_per_player = stats_df.groupby('player_id')['aces'].mean()
        return aces_per_player.idxmax(), aces_per_player.max()
    else:
        return "Required columns not found in stats_df."

# Q9: Player with highest average double faults per match
def q9(stats_df):
    if 'player_id' in stats_df.columns and 'double_faults' in stats_df.columns:
        df_per_player = stats_df.groupby('player_id')['double_faults'].mean()
        return df_per_player.idxmax(), df_per_player.max()
    else:
        return "Required columns not found in stats_df."

# Q10: Correlation between first serve percentage and match win
def q10(stats_df):
    if {'first_serve_pct', 'match_winner'}.issubset(stats_df.columns):
        return stats_df[['first_serve_pct', 'match_winner']].corr()
    else:
        return "Required columns not found in stats_df."

# Q11: Average number of break points saved per match
def q11(stats_df):
    if 'break_points_saved' in stats_df.columns:
        return stats_df['break_points_saved'].mean()
    else:
        return "Column 'break_points_saved' not found in stats_df."

# Q12: Distribution of matches by tournament
def q12(matches_df):
    return matches_df['tourney_name'].value_counts()

# Q13: Player with highest win percentage
def q13(matches_df):
    wins = matches_df['winner_name'].value_counts()
    losses = matches_df['loser_name'].value_counts()
    total_matches = wins.add(losses, fill_value=0)
    win_percentage = (wins / total_matches).fillna(0)
    return win_percentage.idxmax(), win_percentage.max()

# Q14: Average number of sets per match
def q14(matches_df):
    if 'score' in matches_df.columns:
        return matches_df['score'].apply(lambda x: len(str(x).split())).mean()
    else:
        return "Column 'score' not found in matches_df."

# Q15: Most common match score
def q15(matches_df):
    if 'score' in matches_df.columns:
        return matches_df['score'].value_counts().idxmax()
    else:
        return "Column 'score' not found in matches_df."

# Q16: Player with most matches played
def q16(matches_df):
    players = pd.concat([matches_df['winner_name'], matches_df['loser_name']])
    return players.value_counts().idxmax(), players.value_counts().max()

# Q17: Year with most matches played
def q17(matches_df):
    if 'tourney_date' in matches_df.columns:
        matches_df['year'] = pd.to_datetime(matches_df['tourney_date'], errors='coerce').dt.year
        return matches_df['year'].value_counts().idxmax()
    else:
        return "Column 'tourney_date' not found in matches_df."


In [ ]:
print("Q1: Total number of matches:", q1(matches_df))
print("Q2: Number of unique players:", q2(matches_df))
print("Q3: Top 5 players with most wins:\n", q3(matches_df))
print("Q4: Top 5 players with most losses:\n", q4(matches_df))
print("Q5: Average match duration:", q5(matches_df))
print("Q6: Surface distribution of matches:\n", q6(matches_df))
print("Q7: Distribution of match outcomes by tournament level:\n", q7(matches_df))
print("Q8: Player with highest average aces per match:", q8(stats_df))
print("Q9: Player with highest average double faults per match:", q9(stats_df))
print("Q10: Correlation between first serve percentage and match win:\n", q10(stats_df))
print("Q11: Average number of break points saved per match:", q11(stats_df))
print("Q12: Distribution of matches by tournament:\n", q12(matches_df))
print("Q13: Player with highest win percentage:", q13(matches_df))
print("Q14: Average number of sets per match:", q14(matches_df))
print("Q15: Most common match score:", q15(matches_df))
print("Q16: Player with most matches played:", q16(matches_df))
print("Q17: Year with most matches played:", q17(matches_df))


### 3. Which player has the highest number of wins?

In [ ]:

# Count wins by player2 losing
wins = matches_df['winner_name'].value_counts()
top_winner = wins.idxmax()
top_wins = wins.max()
print(f"Top player with most wins: {top_winner} ({top_wins} wins)")


### 4. What is the longest match recorded in terms of duration?

In [ ]:

if 'match_duration' in matches_df.columns:
    longest_match = matches_df.loc[matches_df['match_duration'].idxmax()]
    print(f"Longest match: {longest_match['match_duration']} minutes
Match ID: {longest_match['match_id']}")
else:
    print("match_duration column not found in matches_df.")


### 5. How many sets are typically played in a tennis match?

In [ ]:

if 'num_sets' in matches_df.columns:
    print(f"Average number of sets: {matches_df['num_sets'].mean():.2f}")
else:
    print("num_sets column not available.")


### 6. Which country has produced the most successful tennis players?

In [ ]:

if 'player_country' in power_df.columns:
    top_country = power_df['player_country'].value_counts().idxmax()
    print(f"Country with most successful players: {top_country}")
else:
    print("player_country column not found.")


### 7. What is the average number of aces per match?

In [ ]:

if 'aces' in stats_df.columns:
    avg_aces = stats_df['aces'].mean()
    print(f"Average number of aces per match: {avg_aces:.2f}")
else:
    print("aces column not found.")


### 8. Is there a difference in the number of double faults based on gender?

In [ ]:

if 'double_faults' in stats_df.columns and 'gender' in stats_df.columns:
    print(stats_df.groupby('gender')['double_faults'].mean())
else:
    print("Required columns not available.")


### 9. Which player has won the most tournaments in a single month?

In [ ]:

matches_df['tourney_month'] = pd.to_datetime(matches_df['tourney_date'], errors='coerce').dt.month
monthly_wins = matches_df.groupby(['winner_name', 'tourney_month']).size().reset_index(name='wins')
top_entry = monthly_wins.loc[monthly_wins['wins'].idxmax()]
print(f"Player with most wins in a single month: {top_entry['winner_name']} ({top_entry['wins']} wins in month {int(top_entry['tourney_month'])})")


### 10. Is there a correlation between a player's height and their ranking?

In [ ]:

if 'player_height' in power_df.columns and 'player_rank' in power_df.columns:
    corr = power_df[['player_height', 'player_rank']].dropna().corr().iloc[0, 1]
    print(f"Correlation between height and ranking: {corr:.2f}")
else:
    print("Required columns not found.")


### 11. What is the average duration of matches?

In [ ]:

if 'match_duration' in matches_df.columns:
    print(f"Average match duration: {matches_df['match_duration'].mean():.2f} minutes")
else:
    print("match_duration column not found.")


### 12. What is the average number of games per set in men's vs. women's matches?

In [ ]:

if 'winner_sets_won' in stats_df.columns and 'loser_sets_won' in stats_df.columns and 'gender' in stats_df.columns:
    stats_df['total_sets'] = stats_df['winner_sets_won'] + stats_df['loser_sets_won']
    stats_df['games_per_set'] = stats_df['total_games'] / stats_df['total_sets']
    print(stats_df.groupby('gender')['games_per_set'].mean())
else:
    print("Required columns not available.")


### 13. What is the distribution of left-handed versus right-handed players?

In [ ]:

if 'hand' in power_df.columns:
    print(power_df['hand'].value_counts())
else:
    print("hand column not found.")


### 14. What is the most common type of surface used in tournaments?

In [ ]:

if 'surface' in matches_df.columns:
    print(matches_df['surface'].value_counts().idxmax())
else:
    print("surface column not found.")


### 15. How many distinct countries are represented in the dataset?

In [ ]:

if 'player_country' in power_df.columns:
    print(f"Distinct countries: {power_df['player_country'].nunique()}")
else:
    print("player_country column not found.")


### 16. Which player has the highest winning percentage against top 10 ranked opponents?

In [ ]:

# Simplified assumption: top 10 opponent defined via opponent_rank column in stats_df
if 'opponent_rank' in stats_df.columns and 'winner_name' in stats_df.columns:
    top_10_matches = stats_df[stats_df['opponent_rank'] <= 10]
    win_pct = top_10_matches.groupby('winner_name').size() / top_10_matches['opponent_name'].value_counts()
    win_pct = win_pct.dropna().sort_values(ascending=False)
    print(win_pct.head(1))
else:
    print("Required columns not found.")


### 17. What is the average number of breaks of serve per match?

In [ ]:

if 'break_points_converted' in stats_df.columns:
    print(f"Average breaks per match: {stats_df['break_points_converted'].mean():.2f}")
else:
    print("break_points_converted column not found.")
